all the imports

In [1]:
import arxiv
import requests
import json
from functools import lru_cache
import logging
import chromadb
from sentence_transformers import SentenceTransformer
import gradio as grad
import google.generativeai as genai


c:\Users\Lenovo\.conda\envs\r-conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
@lru_cache(maxsize=100)


def get_arxiv_papers(query: str, max_papers: int = 5) -> list[dict]:
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results = max_papers)
    papers = []

    for result in client.results(search):
        paper = {
            "title" : result.title,
            "authors": [a.name for a in result.authors],
            "summary": result.summary,
            "published": str(result.published.date()),
            "url": result.pdf_url
        }
        papers.append(paper)

    return papers

In [4]:
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(5))


def get_ieee_papers(query:str, api_key:str, max_results:int = 5) -> list[dict]:
    url = "https://ieeexploreapi.ieee.org/api/v1/search/articles"
    params = {
        "querytext": query,
        'apikey': api_key,
        'max_records': max_results,
        'format': 'json'
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        papers = []
        
        for article in response.json().get('articles', []):
            papers.append({
                "title": article.get('title', 'no title'),
                "authors": [author["full_name"] for author in article.get('authors', [])],
                "abstract":article.get('abstract', 'No abstract'),
                'published': article.get('publication_date',""),
                'url': article.get('pdf_url', article.get('html_url',''))
            })
        return papers
    except Exception as e:
        raise Exception(f'IEEE API request failed: {str(e)}')

In [5]:
def research_agent(query:str, ieee_api_key:str, max_papers: int = 5)-> list[str]:
    try:
        papers = get_arxiv_papers(query, max_papers)
        if papers:
            return papers
    except Exception as e:
        print(f'arXiv failed: {str(e)}')
        
    if ieee_api_key:
        try:
            return get_ieee_papers(query, ieee_api_key, max_papers)
        except Exception as e:
            print(f'IEEE failed: {str(e)}')
    raise Exception("something's wrong with API connection")        



In [6]:
def get_results(papers: list[dict]):
    for i, paper in enumerate(papers,1):
        print(f"""
        [{i}] {paper['title']}
        Authors:{''.join(paper.get('authors', []))}
        Published: {paper.get('published', 'N/A')}
        Summary: {paper.get('summary', paper.get('abstract', 'N/A'))[:500]}...
        URL:{paper.get('url','N/A')}
        """)

In [7]:
papers = research_agent('LLM quantization', ieee_api_key="#")
get_results(papers)

INFO:arxiv:Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=LLM+quantization&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
INFO:arxiv:Got first page: 100 of 70131 total results



        [1] A Comprehensive Evaluation of Quantization Strategies for Large Language Models
        Authors:Renren JinJiangcun DuWuwei HuangWei LiuJian LuanBin WangDeyi Xiong
        Published: 2024-02-26
        Summary: Increasing the number of parameters in large language models (LLMs) usually
improves performance in downstream tasks but raises compute and memory costs,
making deployment difficult in resource-limited settings. Quantization
techniques, which reduce the bits needed for model weights or activations with
minimal performance loss, have become popular due to the rise of LLMs. However,
most quantization studies use pre-trained LLMs, and the impact of quantization
on instruction-tuned LLMs and the rel...
        URL:http://arxiv.org/pdf/2402.16775v2
        

        [2] Achieving binary weight and activation for LLMs using Post-Training Quantization
        Authors:Siqing SongChuang WangRuiqi WangYi YangXuyao Zhang
        Published: 2025-04-07
        Summary: Quantizing

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

client = chromadb.Client()
collection = client.create_collection('research_papers')

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [9]:
def store_papers(papers: list[dict]):
    ids = [paper['url'] for paper in papers]
    documents = [paper['title']+ '\n' + paper['summary'] for paper in papers]
    embeddings = model.encode(documents).tolist()

    metadatas = []
    for paper in papers:
        metadata = paper.copy()
        metadata['authors'] = ','.join(paper['authors'])
        metadata['published'] = str(paper['published'])
        metadatas.append(metadata)

    collection.add(
        ids = ids,
        embeddings = embeddings,
        documents = documents,
        metadatas = metadatas
    )

def similar_papers(query:str, n_results:int = 5) -> list[dict]:
    results = collection.query(
        query_embeddings = model.encode(query).tolist(),
        n_results = n_results
    )
    return results['metadatas'][0]

In [10]:
papers = research_agent('Quant Finance', ieee_api_key='#')

store_papers(papers)

relevant_papers = similar_papers('Price Prediction techniques')

get_results(relevant_papers)


INFO:arxiv:Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=Quant+Finance&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
INFO:arxiv:Requesting page (first: True, try: 1): https://export.arxiv.org/api/query?search_query=Quant+Finance&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
INFO:arxiv:Got first page: 100 of 30847 total results
Batches: 100%|██████████| 1/1 [00:00<00:00, 187.74it/s]


        [1] Quant 4.0: Engineering Quantitative Investment with Automated, Explainable and Knowledge-driven Artificial Intelligence
        Authors:Jian Guo,Saizhuo Wang,Lionel M. Ni,Heung-Yeung Shum
        Published: 2022-12-13
        Summary: Quantitative investment (``quant'') is an interdisciplinary field combining
financial engineering, computer science, mathematics, statistics, etc. Quant
has become one of the mainstream investment methodologies over the past
decades, and has experienced three generations: Quant 1.0, trading by
mathematical modeling to discover mis-priced assets in markets; Quant 2.0,
shifting quant research pipeline from small ``strategy workshops'' to large
``alpha factories''; Quant 3.0, applying deep learning...
        URL:http://arxiv.org/pdf/2301.04020v1
        

        [2] Quant GANs: Deep Generation of Financial Time Series
        Authors:Magnus Wiese,Robert Knobloch,Ralf Korn,Peter Kretschmer
        Published: 2019-07-15
        Summary: Modeling

In [11]:
def gradio(query: str):
    papers = research_agent(query, ieee_api_key='#')

    store_papers(papers)

    output = []

    for i, paper in enumerate(papers,1):
        output.append(
            f'**[{i}] {paper['title']}**\n'
            f'Authors: {','.join(paper['authors'])}\n'
            f'Published: {paper['published']}\n'
            f'Summary: {paper['summary'][:500]}\n'
            f'[Read Paper]({paper['url']})\n'
        )

    return "\n\n".join(output)

grad.Interface(
    fn = gradio,
    inputs = grad.Textbox(label="Research Topic", placeholder="e.g., Explainable AI"),
    outputs=grad.Markdown(label='Results'),
    title= "Autonomous Research Agent",
    description="Fetch tech papers from arXiv with semantic search"
).launch(share= True)

INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://a0c99e0bc6b4bd3571.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://a0c99e0bc6b4bd3571.gradio.live "HTTP/1.1 200 OK"


gemini connection

In [12]:
genai.configure(api_key="#")

model = genai.GenerativeModel('gemini-pro')


In [13]:
def suggest(papers):
    prompt = """You are a helpful AI assistant that helps researchers find good project topics.
Below are the titles and abstracts of recent research papers. Based on these, generate 3–5 new, actionable, and specific research ideas.

For each idea, use this format:
---
Title: [Short title]
Problem Statement: [What the research would explore]
Motivation: [Why this problem matters]
Suggested Approach: [Methodology / tools / techniques]
Possible Datasets: [If any]
Tags: [Domain, difficulty (Beginner/Intermediate/Advanced), novelty (High/Medium/Low)]
Inspired by: [Which paper(s) this idea builds on]

Research Papers:
"""

    for i, paper in enumerate(papers, 1):
        prompt += f"\nPaper {i}:\nTitle: {paper['title']}\nAbstract: {paper['abstract']}\n"

    prompt += "\nNow generate the ideas.\n"

    response = model.generate_content(prompt)
    return response.text
